# Make clean data

In [140]:
import pandas as pd

bubbleList = []
countingList = []
misPredList = []
mergeList = []
quickList = []
readCSVList = []
pcList = ["6", "16", "21", "23"]

# quickList.append(pd.read_csv("SummerResults/quickSort-400runs-PC16.csv"))
# quickList[0].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]

# bubbleList.append(pd.read_csv("SummerResults/bubbleSort-400runs-PC16.csv"))
# bubbleList[0].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]

# readCSVList.append(pd.read_csv("SummerResults/readCSV-PC16.csv"))
# readCSVList[0].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]

misPredList.append(pd.read_csv("SummerResults/readCSV-PC16.csv").iloc[47061:,:])
misPredList[0].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]

misPredList[0]


,Test name,PKG (J),CPU (J),GPU (J),DRAM (J),Temp (C),Time (ms)
47061,branchMispredict0-10000-1,1.527954,1.302734,,0.089539,71,165.994
47062,branchMispredict0-10000-1,1.513000,1.283508,,0.091003,71,163.775
47063,branchMispredict0-10000-1,1.475037,1.223511,,0.097595,71,196.106
47064,branchMispredict0-10000-1,1.531860,1.313965,,0.085999,71,152.866
47065,branchMispredict0-10000-1,1.538635,1.308411,,0.088501,71,170.760
...,...,...,...,...,...,...,...
49054,branchMispredict0-50000-1,2.360718,2.058655,,0.139038,67,222.973
49055,branchMispredict0-50000-1,2.282593,1.948608,,0.139587,67,258.803
49056,branchMispredict0-50000-1,2.243652,1.946899,,0.130615,67,227.170
49057,branchMispredict0-50000-1,2.350708,2.029419,,0.140137,67,232.513


In [141]:
def removeNonsense(df):
    df = df[df["PKG (J)"]>=0]
    df = df[df["CPU (J)"]>=0]
    df = df[df["Time (ms)"]>=0]
    return df

In [142]:
def makeinputColumn(df):
    l = []
    for i in range(len(df)):
        l.append(df.iloc[i]["Test name"].split("-")[0])
        print()
        

    df["Input case"] = l

def makeSizecolumn(df):
    l = []
    for i in range(len(df)):
        l.append(int(df.iloc[i]["Test name"].split("-")[1]))

    df["Size"] = l

In [143]:
def makeinputColumn_readCSV(df):
    l = []
    for i in range(len(df)):
        l.append(df.iloc[i]["Test name"].split("-")[1].lower())
    df["Input case"] = l


def makeSizecolumn_readCSV(df):
    l = []
    for i in range(len(df)):
        l.append(int(df.iloc[i]["Test name"].split("-")[-1]))

    df["Size"] = l

In [144]:
def removeOutliers(df, type_, size, col):
    q1 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.25)
    q3 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.75)
    IQR = q3 - q1 
    LF = q1 - 1.5 * IQR
    UF = q3 + 1.5 * IQR

    df1 = df[df["Input case"] == type_][df["Size"]==size][df[col] <= UF]
    df2 = df1[df1["Input case"] == type_][df1["Size"]==size][df1[col] >= LF]
    return df2[col]


In [145]:
def CleanDataframe(df, DataOrBase):
    if DataOrBase == "data":
        cases = ["best", "random", "worst"]
    elif DataOrBase == "misPred":
        cases = ["greaterThan0", "greaterThanEquals0"]
    elif DataOrBase == "misPredCSV":
        cases = ["branchMispredict0"]
    else:
        cases = ["random", "reversesorted", "sorted"]

    cleanList = []
    counter = 0
    #print(df["Input case"].unique())
    
    for case in cases:
        if case == "greaterThanEquals0" or case == "greaterThan0" or case == "branchMispredict0":
            for size in [10000,20000,30000,40000,50000]:
                cleanPortion = pd.DataFrame({"PKG (J)":[]})

                for col in ["PKG (J)", "CPU (J)", "DRAM (J)","Time (ms)", "Temp (C)"]:
                    counter += 1
                    cleanPortion[col] = removeOutliers(df, case, size, col)
                
                cleanPortion["Input case"] = [case for _ in range(len(cleanPortion))]
                cleanPortion["Size"] = [int(size) for _ in range(len(cleanPortion))]

                cleanList.append(cleanPortion)
        elif case != "random" and case != "countingrandom":
            for size in [25000,50000,75000,100000,125000,150000,175000,200000]:
                cleanPortion = pd.DataFrame({"PKG (J)":[]})
                for col in ["PKG (J)", "CPU (J)", "DRAM (J)","Time (ms)", "Temp (C)"]:
                    cleanPortion[col] = removeOutliers(df, case, size, col)
                
                cleanPortion["Input case"] = [case for _ in range(len(cleanPortion))]
                cleanPortion["Size"] = [int(size) for _ in range(len(cleanPortion))]

                cleanList.append(cleanPortion)
        else:
            for i in range(1,11):
                for size in [25000,50000,75000,100000,125000,150000,175000,200000]:
                    cleanPortion = pd.DataFrame({"PKG (J)":[]})
                    for col in ["PKG (J)", "CPU (J)", "DRAM (J)","Time (ms)", "Temp (C)"]:
                        cleanPortion[col] = removeOutliers(df, case+str(i), size, col)
                
                    cleanPortion["Input case"] = [case+str(i) for _ in range(len(cleanPortion))]
                    cleanPortion["Size"] = [int(size) for _ in range(len(cleanPortion))]
                    cleanList.append(cleanPortion)
    #print("counter: ", counter)
    cleanBubble = pd.concat(cleanList)
    return cleanBubble

In [146]:
def prepareDataframe(df, DataOrBase):
    df = removeNonsense(df)

    if DataOrBase == "data" or DataOrBase == "misPred" or DataOrBase == "misPredCSV":
        makeinputColumn(df)
        makeSizecolumn(df)
    else:
        makeinputColumn_readCSV(df)
        makeSizecolumn_readCSV(df)    

    return df


In [147]:
def CleanCSV (df, DataOrBase, name, pcNO):
    ready = prepareDataframe(df, DataOrBase)
    clean = CleanDataframe(ready, DataOrBase).dropna().reset_index(drop=True)
    display(clean)
    clean.to_csv("SummerAnalysis\\"+name+"Clean-PC"+pcNO+".csv", sep=",", index=False)
    return clean

In [148]:

#CleanCSV(quickList[0], "data","QucikSlow-400runs",pcList[1])
# CleanCSV(readCSVList[0], "base","readCSVSlow",pcList[1])

#CleanCSV(bubbleList[0], "data","Bubble-400runs",pcList[1])

df = CleanCSV(misPredList[0], "misPredCSV","BranchMisPredictionREADCSV",pcList[1])

C:\Users\langk\AppData\Local\Temp\ipykernel_15364\2727353117.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[df["Input case"] == type_][df["Size"]==size][df[col] <= UF]
C:\Users\langk\AppData\Local\Temp\ipykernel_15364\2727353117.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[df["Input case"] == type_][df["Size"]==size][df[col] <= UF]
C:\Users\langk\AppData\Local\Temp\ipykernel_15364\2727353117.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[df["Input case"] == type_][df["Size"]==size][df[col] <= UF]
C:\Users\langk\AppData\Local\Temp\ipykernel_15364\2727353117.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[df["Input case"] == type_][df["Size"]==size][df[col] <= UF]
C:\Users\langk\AppData\Local\Temp\ipykernel_15364\2727353117.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
 

,PKG (J),CPU (J),DRAM (J),Time (ms),Temp (C),Input case,Size
0,1.483276,1.278564,0.081726,148.736,69.0,branchMispredict0,10000
1,1.434998,1.235535,0.081299,138.324,69.0,branchMispredict0,10000
2,1.543823,1.301941,0.091492,174.954,69.0,branchMispredict0,10000
3,1.509460,1.287048,0.089050,166.672,69.0,branchMispredict0,10000
4,1.516907,1.312866,0.082764,152.392,69.0,branchMispredict0,10000
...,...,...,...,...,...,...,...
1600,2.311707,2.005859,0.137634,227.254,67.0,branchMispredict0,50000
1601,2.360718,2.058655,0.139038,222.973,67.0,branchMispredict0,50000
1602,2.282593,1.948608,0.139587,258.803,67.0,branchMispredict0,50000
1603,2.243652,1.946899,0.130615,227.170,67.0,branchMispredict0,50000


In [149]:

#CleanCSV(bubbleList[0], "data","Bubble",pcList[0])
df["Size"].value_counts()

20000    372
50000    366
10000    334
30000    316
40000    217
Name: Size, dtype: int64